In [2]:
from mica_text_coref.coref.movie_coref import data

import collections
import matplotlib.pyplot as plt
import numpy as np
import os
import string
import torch
from transformers import AutoTokenizer

In [10]:
torch.FloatTensor([True, False, False, True])

tensor([1., 0., 0., 1.])

: 

In [2]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [3]:
corpus = data.CorefCorpus("/home/sbaruah_usc_edu/mica_text_coref/data/"
                          "movie_coref/results/regular/movie.jsonlines")
dataset = data.CharacterRecognitionDataset(
    corpus, tokenizer, seq_length=256, obey_scene_boundaries=False,
    label_type="head")

In [4]:
print(collections.Counter(dataset.label_ids.flatten().tolist()))
print(collections.Counter(dataset.parse_tag_ids.flatten().tolist()))
print(collections.Counter(dataset.pos_tag_ids.flatten().tolist()))
print(collections.Counter(dataset.ner_tag_ids.flatten().tolist()))

Counter({1: 176964, 0: 25788})
Counter({1: 101599, 3: 73447, 2: 11794, 0: 9608, 4: 3637, 5: 2667})
Counter({4: 105652, 0: 54160, 1: 27721, 3: 7846, 2: 7373})
Counter({4: 191855, 1: 5406, 0: 4528, 2: 886, 3: 77})


In [5]:
lowercased_mentions = []
mentions = []
tokens = []
for document in corpus:
    _tokens = document.token
    for _token in _tokens:
        tokens.append(_token.lower())
    for _mentions in document.clusters.values():
        for _mention in _mentions:
            mention = " ".join(_tokens[_mention.begin: _mention.end + 1])
            lowercased_mentions.append(mention.lower())
            mentions.append(mention)

In [6]:
mention_tups = sorted(collections.Counter(mentions).items(),
                      key=lambda x: x[1], reverse=True)
for mention, count in mention_tups:
    print(f"{mention} : {count}")

I : 1934
his : 1549
you : 1387
her : 771
he : 702
He : 668
him : 626
You : 479
me : 420
Angier : 412
John : 366
Borden : 345
HOPPS : 317
ANGIER : 303
she : 301
your : 292
Hopps : 284
She : 277
my : 266
TONY : 223
BORDEN : 221
LEE : 212
MARCUS : 205
we : 191
STEVE : 187
NICK : 187
Cutter : 186
WE : 185
CUTTER : 184
Nick : 181
JOHN : 168
REGAN : 167
NEIL : 165
Neil : 160
EVELYN : 157
CHARLIE : 151
KEATING : 147
SMART HULK : 128
HIS : 120
Bourne : 120
THOR : 119
THANOS : 108
Keating : 107
Todd : 102
KNOX : 101
Charlie : 101
NATASHA : 100
ROCKET : 94
OLIVIA : 90
Knox : 89
Steve : 86
SCOTT LANG : 86
RHODEY : 85
HER : 84
TODD : 84
sir : 81
Tony : 76
himself : 76
We : 76
Olivia : 76
NEBULA : 72
His : 70
Fallon : 70
Judy : 60
CLINT BARTON : 58
Thanos : 58
HIM : 56
CAMERON : 56
VIGGO : 55
HE : 54
My : 54
SARAH : 54
BOGO : 53
RED : 52
Andy : 51
Thor : 50
MEEKS : 49
Chris : 49
Viggo : 48
CHRIS : 47
WICK : 46
MR . NOLAN : 45
Iosef : 45
Scott : 44
Sarah : 44
BOURNE : 44
Cameron : 43
Meeks : 43
STU 

In [7]:
lowercased_mention_tups = sorted(
    collections.Counter(lowercased_mentions).items(), key=lambda x: x[1],
    reverse=True)
for mention, count in lowercased_mention_tups:
    print(f"{mention} : {count}")

i : 1939
you : 1880
his : 1739
he : 1424
her : 891
angier : 717
him : 682
she : 617
hopps : 601
borden : 568
john : 534
we : 452
me : 438
cutter : 371
nick : 368
neil : 325
my : 322
your : 319
tony : 299
steve : 273
keating : 254
charlie : 252
marcus : 222
lee : 213
knox : 190
todd : 186
thor : 169
regan : 167
thanos : 166
olivia : 166
bourne : 164
evelyn : 158
smart hulk : 148
rocket : 121
natasha : 118
viggo : 103
sir : 102
rhodey : 100
cameron : 99
sarah : 98
chris : 96
meeks : 92
andy : 90
nebula : 89
red : 88
scott lang : 87
himself : 83
scott : 81
bogo : 81
iosef : 80
tesla : 77
alley : 76
fallon : 75
clint : 73
mr . nolan : 73
bellwether : 71
judy : 70
vosen : 67
root : 64
pitts : 60
clint barton : 58
lee ' s : 55
mr . perry : 54
landy : 50
julia : 49
wick : 47
lt . hicox : 47
bad nebula : 45
clawhauser : 44
bridget : 44
gamora : 43
stu hopps : 43
aurelio : 42
evelyn ' s : 42
owens : 41
major hellstrom : 38
flash : 37
hirsch : 37
bourne ' s : 36
chet : 35
bonnie hopps : 35
moose

In [8]:
pronouns = "you i he my him me his yourself mine your her she"
for pronoun in pronouns.split():
    x = sum([t == pronoun for t in tokens])
    y = sum([m == pronoun for m in mentions])
    print(f"{pronoun} {y}/{x}")

you 1387/2307
i 5/2009
he 702/1485
my 266/358
him 626/714
me 420/479
his 1549/1801
yourself 17/24
mine 10/14
your 292/411
her 771/919
she 301/635


In [9]:
n_pronouns = sum([m in pronouns.split() for m in mentions])
print(n_pronouns, len(mentions))

6346 25793


In [10]:
n_mentions_with_end_head = 0
n_mentions = 0
for document in corpus:
    for mentions in document.clusters.values():
        for mention in mentions:
            n_mentions += 1
            if mention.end == mention.head:
                n_mentions_with_end_head += 1
print(f"Number of mentions : {n_mentions}")
print(f"Number of mentions w/ end=head : {n_mentions_with_end_head}")
print(f"Percentage = {100*n_mentions_with_end_head/n_mentions:.2f}%")

Number of mentions : 25793
Number of mentions w/ end=head : 25450
Percentage = 98.67%


In [30]:
dev_dir = ("/home/sbaruah_usc_edu/mica_text_coref/data/movie_coref/results/"
           "character_recognition/epoch_9/dev")
print(f"files = {os.listdir(dev_dir)}")
subtoken_ids = torch.load(os.path.join(dev_dir, "subtoken_ids.pt")).tolist()
token_offset = torch.load(os.path.join(dev_dir, "token_offset.pt")).tolist()
attn_mask = torch.load(os.path.join(dev_dir, "attention_mask.pt")).tolist()
labels = torch.load(os.path.join(dev_dir, "labels.pt")).tolist()
logits = torch.load(os.path.join(dev_dir, "logits.pt"))
parse_ids = torch.load(os.path.join(dev_dir, "parse_ids.pt")).tolist()
pos_ids = torch.load(os.path.join(dev_dir, "pos_ids.pt")).tolist()
ner_ids = torch.load(os.path.join(dev_dir, "ner_ids.pt")).tolist()

files = ['ner_ids.pt', 'token_offset.pt', 'attention_mask.pt', 'logits.pt', 'subtoken_ids.pt', 'pos_ids.pt', 'labels.pt', 'parse_ids.pt']


In [31]:
poslabel = 0
true_positives: list[tuple[str, str, str, str]] = []
false_positives: list[tuple[str, str, str, str]] = []
false_negatives: list[tuple[str, str, str, str]] = []

for (_subtoken_ids, _token_offset, _attn_mask, _labels, _logits, _parse_ids,
     _pos_ids, _ner_ids) in zip(
        subtoken_ids, token_offset, attn_mask, labels, logits, parse_ids,
        pos_ids, ner_ids):
    _subtokens = tokenizer.convert_ids_to_tokens(_subtoken_ids)
    _tokens, _parse, _pos, _ner = [], [], [], []
    for (begin, end), __parse_id, __pos_id, __ner_id in zip(
        _token_offset, _parse_ids, _pos_ids, _ner_ids):
        if begin == end == 0:
            break
        _token_subtokens = _subtokens[begin: end + 1]
        _token = tokenizer.convert_tokens_to_string(_token_subtokens).lstrip()
        _tokens.append(_token)
        _parse.append(dataset.parse_tag_set[__parse_id])
        _pos.append(dataset.pos_tag_set[__pos_id])
        _ner.append(dataset.ner_tag_set[__ner_id])
    _pred = _logits.argmax(dim=1).tolist()
    _labels = _labels[:len(_tokens)]
    _pred = _pred[:len(_tokens)]
    for __token, __pred, __label, __parse, __pos, __ner in zip(
        _tokens, _pred, _labels, _parse, _pos, _ner):
        print(f"{__token:20s}\t{__parse}\t{__pos:9s}\t{__ner:6s}\t{__label}"
              f"\t{__pred}")
        if __pred == __label:
            if __label == poslabel:
                true_positives.append((__token, __parse, __pos, __ner))
        else: 
            if __label == poslabel:
                false_negatives.append((__token, __parse, __pos, __ner))
            else:
                false_positives.append((__token, __parse, __pos, __ner))
    print("\n\n")

print(f"true positives = {len(true_positives)}")
print(f"false positives = {len(false_positives)}")
print(f"false negatives = {len(false_negatives)}")
print()

print("true positives:")
for _token, _parse, _pos, _ner in true_positives:
    print(f"{_token:20s}\t{_parse}\t{_pos:9s}\t{_ner:6s}")
print("\n\n")

print("false positives:")
for _token, _parse, _pos, _ner in false_positives:
    print(f"{_token:20s}\t{_parse}\t{_pos:9s}\t{_ner:6s}")
print("\n\n")

print("false negatives:")
for _token, _parse, _pos, _ner in false_negatives:
    print(f"{_token:20s}\t{_parse}\t{_pos:9s}\t{_ner:6s}")

THE                 	<O>	<O>      	<O>   	1	1
SHAWSHANK           	<O>	NOUN     	<O>   	1	0
REDEMPTION          	<O>	NOUN     	<O>   	1	1
by                  	<O>	<O>      	<O>   	1	1
Frank               	C	NOUN     	PERSON	1	0
Darabont            	C	NOUN     	PERSON	1	0
Based               	N	VERB     	<O>   	1	1
upon                	N	<O>      	<O>   	1	1
the                 	N	<O>      	<O>   	1	1
story               	N	NOUN     	<O>   	1	1
Rita                	N	NOUN     	PERSON	1	0
Hayworth            	N	NOUN     	PERSON	1	0
and                 	N	<O>      	<O>   	1	1
Shawshank           	N	NOUN     	<O>   	1	0
Redemption          	N	NOUN     	<O>   	1	1
by                  	D	<O>      	<O>   	1	1
Stephen             	D	NOUN     	PERSON	1	0
King                	D	NOUN     	PERSON	1	1
1                   	S	<O>      	<O>   	1	1
INT                 	S	NOUN     	<O>   	1	1
--                  	S	<O>      	<O>   	1	1
CABIN               	S	NOUN     	<O>   	1	1
--                  	S	<

In [32]:
false_positive_tokens = [x[0] for x in false_positives]
false_negative_tokens = [x[0] for x in false_negatives]
print(sorted(collections.Counter(false_positive_tokens).items(),
             key=lambda x: x[1], reverse=True)[:50])
print(sorted(collections.Counter(false_negative_tokens).items(),
             key=lambda x: x[1], reverse=True)[:50])

[('to', 140), (',', 111), ('t', 100), ('the', 96), ("'", 89), ('and', 82), ('s', 67), (')', 55), ('.', 47), ('LT', 47), ('a', 46), ('CRI', 46), ('we', 36), ('MAJOR', 35), ('it', 34), ('they', 33), ('V', 32), ('ll', 29), ('car', 28), ('They', 26), ('D', 26), ('A', 25), ('you', 24), ('O', 24), ('Bourne', 24), ('The', 23), ('OS', 20), ('S', 19), ('SGT', 19), ('It', 18), ('We', 16), ('name', 16), ('who', 15), ('just', 15), ('agents', 15), ('not', 14), ('d', 14), ('POLA', 14), ('Andy', 13), ('men', 13), ('KING', 13), ('gun', 12), ('place', 12), ('this', 12), ('all', 12), ('"', 12), ('Von', 12), ('will', 12), ('deal', 12), ('can', 11)]
[('his', 160), ('you', 77), ('I', 72), ('him', 71), ('your', 53), ('her', 43), ('me', 35), ('my', 32), ('S', 31), ('You', 29), ('he', 23), ('Red', 23), ('He', 21), ('.', 20), ('s', 19), ('Bourne', 19), ('#', 13), ('His', 9), ('She', 8), ('Sgt', 7), ('HELLSTROM', 7), ('she', 6), ('himself', 6), ('Major', 6), ('Wills', 5), ('we', 4), ('My', 4), ('LEADER', 4), ('

: 